In [32]:
import numpy as np
import pandas as pd
from datetime import datetime
import uszipcode
from uszipcode import SearchEngine

import fredapi as fa

In [2]:
df_car = pd.read_csv('used_cars_data.csv')

/home/diegofvargas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_car.head()

,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [4]:
df_car['main_picture_url'][11425]

'https://static.cargurus.com/images/forsale/2020/08/26/21/29/2017_nissan_rogue-pic-7031015011042504119-152x114.jpeg'

In [5]:
#replace empty values with NaN
df_car = df_car.replace(r'^\s*$', np.nan, regex=True)

In [6]:
(df_car.isnull().sum()/len(df_car)).sort_values(ascending=False)[(df_car.isnull().sum()/len(df_car)).sort_values(ascending=False)>.1]

vehicle_damage_category    1.000000
combine_fuel_economy       1.000000
is_certified               1.000000
bed                        0.993477
cabin                      0.978823
is_oemcpo                  0.954880
is_cpo                     0.939035
bed_height                 0.856969
bed_length                 0.856969
owner_count                0.505664
fleet                      0.475525
theft_title                0.475525
isCab                      0.475525
has_accidents              0.475525
frame_damaged              0.475525
salvage                    0.475525
franchise_make             0.190876
torque                     0.172595
highway_fuel_economy       0.163759
city_fuel_economy          0.163759
power                      0.160473
main_picture_url           0.123029
dtype: float64

In [7]:
#Drop columns that have over 20% of nulls
df_car.drop(['vehicle_damage_category','combine_fuel_economy','is_certified','bed',
             'cabin','is_oemcpo','is_cpo','bed_height','bed_length','owner_count',
             'fleet','theft_title','isCab','has_accidents','frame_damaged','salvage'],
            axis=1, inplace=True)

In [8]:
#Drop columns that don't add predictive power
df_car.drop(['vin','city','description','engine_type','exterior_color','interior_color',
            'franchise_make','listing_id','main_picture_url','major_options','sp_id',
             'sp_name','trimId','trim_name','wheel_system'], axis=1, inplace=True)

In [9]:
df_car.columns

Index(['back_legroom', 'body_type', 'city_fuel_economy', 'daysonmarket',
       'dealer_zip', 'engine_cylinders', 'engine_displacement',
       'franchise_dealer', 'front_legroom', 'fuel_tank_volume', 'fuel_type',
       'height', 'highway_fuel_economy', 'horsepower', 'is_new', 'latitude',
       'length', 'listed_date', 'listing_color', 'longitude', 'make_name',
       'maximum_seating', 'mileage', 'model_name', 'power', 'price',
       'savings_amount', 'seller_rating', 'torque', 'transmission',
       'transmission_display', 'wheel_system_display', 'wheelbase', 'width',
       'year'],
      dtype='object')

In [14]:
df_car.model_name.value_counts()[:60]

F-150                 128586
Silverado 1500         73221
1500                   69117
Equinox                63341
Escape                 56905
Explorer               49001
Rogue                  48470
Grand Cherokee         45602
Camry                  43618
CR-V                   43409
Fusion                 41650
Altima                 38791
Accord                 38084
Civic                  37985
Trax                   37608
Corolla                36478
Cherokee               31703
RAV4                   30891
Tucson                 30022
Malibu                 30004
Edge                   29274
Sentra                 27917
Wrangler Unlimited     27151
Traverse               26887
CX-5                   26658
Highlander             26433
Elantra                26204
Sierra 1500            26109
Terrain                23547
Sonata                 23216
Compass                23128
Jetta                  22470
Pilot                  22395
Expedition             21527
Grand Caravan 

In [15]:
#Create numerical values out of strings
df_car['back_legroom'] = df_car['back_legroom'].replace(r'--', np.nan)
df_car['back_legroom'] = df_car['back_legroom'].apply(lambda x: float(x[:-3]) if isinstance(x, str) else x)

In [16]:
#Create numerical values out of strings
df_car['front_legroom'] = df_car['front_legroom'].replace(r'--', np.nan)
df_car['front_legroom'] = df_car['front_legroom'].apply(lambda x: float(x[:-3]) if isinstance(x, str) else x)

In [17]:
#Create numerical values out of strings
df_car['fuel_tank_volume'] = df_car['fuel_tank_volume'].replace(r'--', np.nan)
df_car['fuel_tank_volume'] = df_car['fuel_tank_volume'].apply(lambda x: float(x[:-3]) if isinstance(x, str) else x)

In [18]:
#Create numerical values out of strings
df_car['height'] = df_car['height'].replace(r'--', np.nan)
df_car['height'] = df_car['height'].apply(lambda x: float(x[:-3]) if isinstance(x, str) else x)

In [19]:
#Create numerical values out of strings
df_car['length'] = df_car['length'].replace(r'--', np.nan)
df_car['length'] = df_car['length'].apply(lambda x: float(x[:-3]) if isinstance(x, str) else x)

In [20]:
#Create numerical values out of strings
df_car['maximum_seating'] = df_car['maximum_seating'].replace(r'--', np.nan)
df_car['maximum_seating'] = df_car['maximum_seating'].apply(lambda x: float(x[:-6]) if isinstance(x, str) else x)

In [21]:
#Create numerical values out of strings
df_car['wheelbase'] = df_car['wheelbase'].replace(r'--', np.nan)
df_car['wheelbase'] = df_car['wheelbase'].apply(lambda x: float(x[:-3]) if isinstance(x, str) else x)

In [22]:
#Create numerical values out of strings
df_car['width'] = df_car['width'].replace(r'--', np.nan)
df_car['width'] = df_car['width'].apply(lambda x: float(x[:-3]) if isinstance(x, str) else x)

In [23]:
#Some are negative, need to check why
df_car['listing_age'] = df_car.listed_date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d").year) - df_car.year

In [24]:
#Create numerical values out of strings
df_car['power'] = df_car['power'].replace(r'--', np.nan)
df_car['power'] = df_car['power'].apply(lambda x: int(x.replace(',', '').split(' hp')[0]) if isinstance(x, str) else x)

In [25]:
#Create numerical values out of strings
df_car['torque'] = df_car['torque'].replace(r'--', np.nan)
df_car['torque'] = df_car['torque'].apply(lambda x: int(x.replace(',', '').split(' lb-ft')[0]) if isinstance(x, str) else x)

In [36]:
#Code to convert zip code to states and then unto region, to cut down on dimensionality

In [33]:
search = SearchEngine()

Download /home/diegofvargas/.uszipcode/simple_db.sqlite from https://github.com/MacHu-GWU/uszipcode-project/releases/download/1.0.1.db/simple_db.sqlite ...
  1.00 MB downloaded ...
  2.00 MB downloaded ...
  3.00 MB downloaded ...
  4.00 MB downloaded ...
  5.00 MB downloaded ...
  6.00 MB downloaded ...
  7.00 MB downloaded ...
  8.00 MB downloaded ...
  9.00 MB downloaded ...
  10.00 MB downloaded ...
  11.00 MB downloaded ...
  Complete!


In [43]:
states_to_regions = {
    'Washington': 'West', 'Oregon': 'West', 'California': 'West', 'Nevada': 'West',
    'Idaho': 'West', 'Montana': 'West', 'Wyoming': 'West', 'Utah': 'West',
    'Colorado': 'West', 'Alaska': 'West', 'Hawaii': 'West', 'Maine': 'Northeast',
    'Vermont': 'Northeast', 'New York': 'Northeast', 'New Hampshire': 'Northeast',
    'Massachusetts': 'Northeast', 'Rhode Island': 'Northeast', 'Connecticut': 'Northeast',
    'New Jersey': 'Northeast', 'Pennsylvania': 'Northeast', 'North Dakota': 'Midwest',
    'South Dakota': 'Midwest', 'Nebraska': 'Midwest', 'Kansas': 'Midwest',
    'Minnesota': 'Midwest', 'Iowa': 'Midwest', 'Missouri': 'Midwest', 'Wisconsin': 'Midwest',
    'Illinois': 'Midwest', 'Michigan': 'Midwest', 'Indiana': 'Midwest', 'Ohio': 'Midwest',
    'West Virginia': 'South', 'District of Columbia': 'South', 'Maryland': 'South',
    'Virginia': 'South', 'Kentucky': 'South', 'Tennessee': 'South', 'North Carolina': 'South',
    'Mississippi': 'South', 'Arkansas': 'South', 'Louisiana': 'South', 'Alabama': 'South',
    'Georgia': 'South', 'South Carolina': 'South', 'Florida': 'South', 'Delaware': 'South',
    'Arizona': 'Southwest', 'New Mexico': 'Southwest', 'Oklahoma': 'Southwest',
    'Texas': 'Southwest', 'Puerto Rico':'Puerto Rico'}

In [38]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [54]:
search.by_zipcode(960).state_long

'Puerto Rico'

In [46]:
def state_search(zipcode):
    result = search.by_zipcode(zipcode).state_long
    if  result == None:
        return 'N/A'
    else:
        return result

In [55]:
df_car['dealer_zip'] = df_car['dealer_zip'].apply(lambda x: x if isinstance(x, int) else x[:5])

In [56]:
#Find the state corresponding to a zipcode
df_car['state'] = df_car['dealer_zip'].apply(state_search)

In [57]:
#Find the region corresponding to a state
df_car['region'] = df_car['state'].apply(lambda x: states_to_regions[x])

In [58]:
df_car['region'].value_counts()

South          892419
Midwest        746586
Southwest      465850
West           452338
Northeast      442743
Puerto Rico       104
Name: region, dtype: int64

In [61]:
#Drop all location related columns, only keeping region
df_car.drop(['dealer_zip','state','latitude','longitude'], axis=1, inplace=True)

In [62]:
df_car.columns

Index(['back_legroom', 'body_type', 'city_fuel_economy', 'daysonmarket',
       'engine_cylinders', 'engine_displacement', 'franchise_dealer',
       'front_legroom', 'fuel_tank_volume', 'fuel_type', 'height',
       'highway_fuel_economy', 'horsepower', 'is_new', 'length', 'listed_date',
       'listing_color', 'make_name', 'maximum_seating', 'mileage',
       'model_name', 'power', 'price', 'savings_amount', 'seller_rating',
       'torque', 'transmission', 'transmission_display',
       'wheel_system_display', 'wheelbase', 'width', 'year', 'listing_age',
       'region'],
      dtype='object')

In [ ]:
#Keeping listing color and eliminating exterior and interior color columns
#Major options has lots of features, but dropping for now
#Keeping make_name instead of franchise_make because it is more percise
#Figure how to deal with car models

In [63]:
#converting listed_date to datetime
df_car['listed_date'] = pd.to_datetime(df_car['listed_date'])

In [64]:
#convert to string and change day to 1 to match output from FRED
df_car['listed_date'] = df_car['listed_date'].apply(lambda x: x.replace(day=1).strftime('%Y-%m-%d'))

In [72]:
df_car.listed_date.value_counts()[:60]/len(df_car)

2020-08-01    0.362917
2020-09-01    0.180426
2020-07-01    0.175458
2020-06-01    0.073050
2020-03-01    0.037126
2020-02-01    0.030440
2020-05-01    0.027651
2020-01-01    0.022258
2020-04-01    0.019710
2019-12-01    0.018234
2019-11-01    0.014104
2019-10-01    0.010793
2019-09-01    0.007459
2019-08-01    0.005451
2019-07-01    0.002999
2019-06-01    0.001980
2019-05-01    0.001643
2019-03-01    0.001249
2019-04-01    0.001216
2019-02-01    0.000740
2019-01-01    0.000722
2018-12-01    0.000660
2018-11-01    0.000523
2018-10-01    0.000451
2018-09-01    0.000364
2018-08-01    0.000328
2018-07-01    0.000255
2018-06-01    0.000217
2018-05-01    0.000170
2018-03-01    0.000145
2018-04-01    0.000132
2017-09-01    0.000103
2018-02-01    0.000096
2017-12-01    0.000084
2018-01-01    0.000081
2017-08-01    0.000071
2017-10-01    0.000070
2017-11-01    0.000051
2017-06-01    0.000043
2016-10-01    0.000043
2017-05-01    0.000039
2017-04-01    0.000038
2015-01-01    0.000034
2017-03-01 

In [15]:
#Fred Data
fred = fa.Fred(api_key='2268954bdb0755419c205b75e2c59d3a')

In [16]:
gdp = fred.get_series('GDP')
gdp.name = 'gdp'

In [21]:
gdp.asfreq('MS', method='bfill').to_period('M')

1946-01          NaN
1946-02          NaN
1946-03          NaN
1946-04          NaN
1946-05          NaN
             ...    
2021-12    24740.480
2022-01    24740.480
2022-02    25248.476
2022-03    25248.476
2022-04    25248.476
Freq: M, Name: gdp, Length: 916, dtype: float64